# Pre-training con datos externos

### Comprobamos la GPU asignada

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! nvidia-smi

Fri May 26 20:52:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Autenticación en HF (para compartir modelo)

Para poder compartir nuestro modelo con la comunidad y generar resultados como el que se muestra en la imagen de arriba a través de la API de inferencia, hay algunos pasos más a seguir.

Primero debe almacenar su token de autenticación del sitio web HuggingFace (regístrate [aquí] (https://huggingface.co/join) si aún no lo ha hecho), luego ejecute la siguiente celda e ingrese su nombre de usuario y contraseña:

In [ ]:
!apt install git-lfs
!git config --global user.email "sgonzalezsilot@gmail.com"
!git config --global user.name "Santiago"
! pip install -q transformers datasets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Elegimos un modelo (checkpoint)

In [ ]:
model_checkpoint = "finiteautomata/beto-emotion-analysis"

In [ ]:
import pandas as pd
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweets_politica_kaggle.csv", sep='\t' ,lineterminator='\r')
df = pd.read_table('/content/drive/MyDrive/Colab Notebooks/tweets_politica_kaggle.csv', sep='\t', header=0, encoding= "utf-8")

In [ ]:
df

,cuenta,partido,timestamp,tweet
0,a34133350b0605cb24081843f63176ca,psoe,1.363973e+09,@vesteve3 @manubenas @ccoo_rm @desobediencia_ ...
1,a34133350b0605cb24081843f63176ca,psoe,1.364061e+09,"“@kirovast: @Hugo_Moran muy fan de la ""radical..."
2,a34133350b0605cb24081843f63176ca,psoe,1.364117e+09,@ALTAS_PRESIONES Nuevos dueños para las renova...
3,a34133350b0605cb24081843f63176ca,psoe,1.364121e+09,@jumanjisolar @solartradex @josea_dolera El di...
4,a34133350b0605cb24081843f63176ca,psoe,1.364153e+09,“@cesarnayu: https://t.co/J4OTXj1x7w … Por fav...
...,...,...,...,...
245785,2a5fcd1034beb5bd30bf5a1528008d81,psoe,1.679572e+09,Un placer inaugurar la jornada #NextGeneration...
245786,561b1a4d4369a0116ac5eba469f28bc0,psoe,1.679572e+09,Y esto también es el PP https://t.co/CaOTdTWnXB
245787,42acd1f0c1c51732519ad0d533573359,pp,1.679572e+09,Actualización: Los 3 violadores ya han sido pu...
245788,747c68ab573c6e4ad8f417f71614e022,pp,1.679572e+09,Con la firma del Convenio de colaboración entr...


In [ ]:
!pip install tweet-preprocessor
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import preprocessor as p
import unidecode
from tqdm import tqdm


# Configuramos tweet-preprocessor para que nos elimine del tweet: Urls, menciones, emoticonos, etc.
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.EMOJI, p.OPT.SMILEY)

# df = df_train
# Limpiamos los tweets
for i in tqdm(range(df.shape[0])):
    # Limpiamos el tweet con tweet-preprocessor y lo pasamos a minúsculas
    df.loc[i, 'clean_tweet'] = (p.clean(unidecode.unidecode(df.iloc[i]['tweet'])))

# df_train = df

100%|██████████| 245790/245790 [02:08<00:00, 1916.10it/s]


In [ ]:
df

,cuenta,partido,timestamp,tweet,clean_tweet
0,a34133350b0605cb24081843f63176ca,psoe,1.363973e+09,@vesteve3 @manubenas @ccoo_rm @desobediencia_ ...,(Buen animo para esta primavera que iniciamos).
1,a34133350b0605cb24081843f63176ca,psoe,1.364061e+09,"“@kirovast: @Hugo_Moran muy fan de la ""radical...",""": muy fan de la ""radicalidad social"""" (Frente..."
2,a34133350b0605cb24081843f63176ca,psoe,1.364117e+09,@ALTAS_PRESIONES Nuevos dueños para las renova...,Nuevos duenos para las renovables. En ese mome...
3,a34133350b0605cb24081843f63176ca,psoe,1.364121e+09,@jumanjisolar @solartradex @josea_dolera El di...,El diferencial de precios energeticos con Alem...
4,a34133350b0605cb24081843f63176ca,psoe,1.364153e+09,“@cesarnayu: https://t.co/J4OTXj1x7w … Por fav...,""": ... Por favor es importante difundir este m..."
...,...,...,...,...,...
245785,2a5fcd1034beb5bd30bf5a1528008d81,psoe,1.679572e+09,Un placer inaugurar la jornada #NextGeneration...,Un placer inaugurar la jornada y compartir las...
245786,561b1a4d4369a0116ac5eba469f28bc0,psoe,1.679572e+09,Y esto también es el PP https://t.co/CaOTdTWnXB,Y esto tambien es el PP
245787,42acd1f0c1c51732519ad0d533573359,pp,1.679572e+09,Actualización: Los 3 violadores ya han sido pu...,Actualizacion: Los 3 violadores ya han sido pu...
245788,747c68ab573c6e4ad8f417f71614e022,pp,1.679572e+09,Con la firma del Convenio de colaboración entr...,Con la firma del Convenio de colaboracion entr...


### Elegimos y pre-procesamos el dataset

In [ ]:
from datasets import load_dataset, Dataset
# dataset = load_dataset('csv', data_files=['/content/drive/MyDrive/Colab Notebooks/tweets_politica_kaggle.csv'], split='train')
dataset = Dataset.from_pandas(df, split='train')
datasets = dataset.train_test_split(test_size=0.10)

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['cuenta', 'partido', 'timestamp', 'tweet', 'clean_tweet'],
        num_rows: 221211
    })
    test: Dataset({
        features: ['cuenta', 'partido', 'timestamp', 'tweet', 'clean_tweet'],
        num_rows: 24579
    })
})

In [ ]:
datasets["train"][0]

{'cuenta': 'e9c8254b481efbfeec53e19ee2957997',
 'partido': 'psoe',
 'timestamp': 1561659559.0,
 'tweet': 'Enhorabuena a las chicas de la #SelFEM @BaloncestoESP por vuestra primera victoria en el #EuroBasketWomen, y también al equipo de Ucrania, que han sido unas dignas rivales.  #QueremosMás #SomosEquipo #ESPasión https://t.co/NDIybejYj1',
 'clean_tweet': 'Enhorabuena a las chicas de la por vuestra primera victoria en el , y tambien al equipo de Ucrania, que han sido unas dignas rivales.'}

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "El número de filas a mostrar no puede ser mayor que el número de filas en el dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,cuenta,partido,timestamp,tweet,clean_tweet
0,a955dd9aa5fa9b46b4e0df5d2ebe8987,pp,1.581536e+09,"📺 Esta noche a partir de las 21:45 os espero en el programa El Faro de @LaOchoTV, presentado por @LuisMotes. Podéis verlo en directo aquí 👇🏼https://t.co/4l5mtdicDu https://t.co/6rtIDy2J2P","Esta noche a partir de las 21:45 os espero en el programa El Faro de , presentado por . Podeis verlo en directo aqui"
1,3a97e533064700b11a5679036006ab12,psoe,1.631216e+09,"Un placer compartir experiencias y opiniones con @ClaudiaLAZamora , presidenta en funciones del senado de #Argentina en el marco del #WCSP https://t.co/ygZP8OjkS4","Un placer compartir experiencias y opiniones con , presidenta en funciones del senado de en el marco del"
2,8e09b5a214b5584a32613ab31772f534,ciudadanos,1.581687e+09,Pocas ideas hay peores en economía que mezclar política y banca. A pesar de ello el Gobierno parece dispuesto a volver a caer en el error de las cajas. Os lo explico en esta nueva edición del #VlogDeGaricano https://t.co/iLdfSYFoOT,Pocas ideas hay peores en economia que mezclar politica y banca. A pesar de ello el Gobierno parece dispuesto a volver a caer en el error de las cajas. Os lo explico en esta nueva edicion del


### Tokenización del dataset

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples["clean_tweet"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_fn, batched=True, remove_columns=["cuenta", "partido", "timestamp", "tweet", "clean_tweet"])

Map:   0%|          | 0/221211 [00:00<?, ? examples/s]

Map:   0%|          | 0/24579 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets.column_names

{'train': ['input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['input_ids', 'token_type_ids', 'attention_mask']}

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [4,
  17142,
  1255,
  29437,
  30933,
  24873,
  30932,
  1454,
  1018,
  1110,
  1108,
  24803,
  30932,
  1036,
  1049,
  13184,
  1096,
  1200,
  1030,
  2399,
  1169,
  1729,
  1017,
  1162,
  17321,
  2969,
  1008,
  2326,
  1481,
  5],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
len(tokenized_datasets["train"][5]['input_ids'])

56

### Formatear el dataset

Ahora, para la parte más difícil: necesitamos concatenar todos nuestros textos juntos y luego dividir el resultado en pequeños trozos de un cierto `block_size`. Para hacer esto, usaremos el método `map` nuevamente, con la opción` batched = True`. Esta opción en realidad nos permite cambiar la cantidad de ejemplos en los conjuntos de datos al devolver una cantidad diferente de ejemplos a la que obtuvimos. De esta manera, podemos crear nuestras nuevas muestras a partir de un lote de ejemplos.

Primero, tomamos la longitud máxima con la que se entrenó previamente nuestro modelo. Esto podría ser demasiado grande para caber en la RAM de nuestra GPU, por lo que aquí tomamos un poco menos de solo 128.

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000)

Map:   0%|          | 0/221211 [00:00<?, ? examples/s]

Map:   0%|          | 0/24579 [00:00<?, ? examples/s]

### Entrenar el modelo

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at finiteautomata/beto-emotion-analysis were not used when initializing BertForMaskedLM: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at finiteautomata/beto-emotion-analysis and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "beto-emotion-twitter-tweets-politica-ES",
    #per_device_train_batch_size = 2,
    #per_device_eval_batch_size = 2,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"beto-emotion-twitter-tweets-politica-ES",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1543: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case sgonzalezsilot/beto-emotion-twitter-tweets-politica-ES).
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    save_strategy = "no"
)

Cloning https://huggingface.co/sgonzalezsilot/beto-emotion-twitter-tweets-politica-ES into local empty directory.


In [ ]:
trainer.evaluate()

{'eval_loss': 11.064772605895996,
 'eval_runtime': 69.5515,
 'eval_samples_per_second': 113.671,
 'eval_steps_per_second': 14.22}

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")